In [1]:
import pandas as pd 
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
import matplotlib.pyplot as plt

In [2]:
dados ='C:/Users/matto/Desktop/Leandro/Mestrado PPCIC/Dissertação/novo tema/Novo cj de dados/Conv1d/Dominio do Tempo/Todos os arrays/dados_tempo.npy'
rotulos ='C:/Users/matto/Desktop/Leandro/Mestrado PPCIC/Dissertação/novo tema/Novo cj de dados/rotulos.npy'

In [3]:
import numpy as np
X = np.load(dados)
y = np.load(rotulos)

In [4]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [5]:
X_train.shape

(1493, 400, 1)

In [6]:
X_train.tolist()
y_train.tolist()
X_test.tolist()
y_test.tolist()



[1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [7]:
batch = int(len(y_train)/30)
batch

49

In [8]:
from keras.utils import np_utils
y_train_convertido = np_utils.to_categorical(y_train)
y_test_convertido =  np_utils.to_categorical(y_test)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout

In [10]:
import optuna
from keras.optimizers import SGD

def objective(trial):
    
    # Definindo o espaço de busca dos hiperparâmetros
    
    num_layers = trial.suggest_int('num_layers', 1, 5)
    dense_neurons = trial.suggest_int('dense_neurons', 20, 4000, log=True)
    dropout = trial.suggest_int('dropout', 2, 5, step=1) / 10.0
    learning_rate = trial.suggest_categorical('learning_rate', [0.001, 0.003,0.005,0.007,0.01,0.03,0.05, 0.07])
    
    modelo = Sequential()
    
    for i in range(num_layers):
        if i == 0:
            modelo.add(Dense(dense_neurons, input_dim=400, kernel_initializer='normal',activation='relu'))
            modelo.add(Dropout(dropout))
        else:
            modelo.add(Dense(dense_neurons, kernel_initializer='normal',activation='relu'))
            modelo.add(Dropout(dropout))

    modelo.add(Dense(2, kernel_initializer='normal',activation = 'softmax'))
                     
    otimizador = SGD(learning_rate=learning_rate) 
    modelo.compile(loss="categorical_crossentropy",optimizer=otimizador,metrics =["acc"])
    historico = modelo.fit(X_train,y_train_convertido, epochs= 300, batch_size = batch, validation_data=(X_test,y_test_convertido),verbose = 1)
    
    return historico.history['val_acc'][-1]

In [11]:
import csv

# criação do objeto de estudo
study = optuna.create_study(direction="maximize")

# abertura do arquivo CSV em modo de escrita
with open("results.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["trial", "accuracy", "hyperparameters"])  # cabeçalho

    # iteração dos trials
    
    study.optimize(objective, n_trials=80)
    best_trial = study.best_trial
    i = study.trials[0].number  # Obtém o número do primeiro trial realizado
    print("Trial {}: Accuracy = {}".format(i, best_trial.value))

    # escrita dos valores no arquivo CSV
    writer.writerow([i, best_trial.value, best_trial.params])

    print("Trial {}: Accuracy = {}".format(i, best_trial.value))


print(study.best_params)



[I 2023-07-08 13:44:04,623] A new study created in memory with name: no-name-5dd5c450-e492-424e-b985-94e3d1d775a9


Epoch 1/300
31/31 [==============================] - 1s 8ms/step - loss: 0.4521 - acc: 0.8480 - val_loss: 0.4178 - val_acc: 0.8471
Epoch 2/300
31/31 [==============================] - 0s 9ms/step - loss: 0.3630 - acc: 0.8727 - val_loss: 0.4368 - val_acc: 0.8471
Epoch 3/300
31/31 [==============================] - 0s 5ms/step - loss: 0.3586 - acc: 0.8727 - val_loss: 0.4001 - val_acc: 0.8471
Epoch 4/300
31/31 [==============================] - 0s 8ms/step - loss: 0.3512 - acc: 0.8727 - val_loss: 0.4312 - val_acc: 0.8471
Epoch 5/300
31/31 [==============================] - 0s 7ms/step - loss: 0.3463 - acc: 0.8727 - val_loss: 0.3776 - val_acc: 0.8471
Epoch 6/300
31/31 [==============================] - 0s 6ms/step - loss: 0.3456 - acc: 0.8727 - val_loss: 0.3650 - val_acc: 0.8471
Epoch 7/300
31/31 [==============================] - 0s 9ms/step - loss: 0.3405 - acc: 0.8747 - val_loss: 0.3708 - val_acc: 0.8471
Epoch 8/300
31/31 [==============================] - 0s 6ms/step - loss: 0.3395 - a

31/31 [==============================] - 0s 8ms/step - loss: 0.2593 - acc: 0.9062 - val_loss: 0.3328 - val_acc: 0.8674
Epoch 64/300
31/31 [==============================] - 0s 6ms/step - loss: 0.2369 - acc: 0.9176 - val_loss: 0.8478 - val_acc: 0.8471
Epoch 65/300
31/31 [==============================] - 0s 8ms/step - loss: 0.2500 - acc: 0.9009 - val_loss: 0.4492 - val_acc: 0.8502
Epoch 66/300
31/31 [==============================] - 0s 7ms/step - loss: 0.2084 - acc: 0.9277 - val_loss: 0.6798 - val_acc: 0.5741
Epoch 67/300
31/31 [==============================] - 0s 7ms/step - loss: 0.2625 - acc: 0.8969 - val_loss: 0.2684 - val_acc: 0.9267
Epoch 68/300
31/31 [==============================] - 0s 9ms/step - loss: 0.1881 - acc: 0.9344 - val_loss: 0.2574 - val_acc: 0.9485
Epoch 69/300
31/31 [==============================] - 0s 8ms/step - loss: 0.2278 - acc: 0.9123 - val_loss: 0.4109 - val_acc: 0.6303
Epoch 70/300
31/31 [==============================] - 0s 9ms/step - loss: 0.2553 - acc: 0

Epoch 125/300
31/31 [==============================] - 0s 5ms/step - loss: 0.1920 - acc: 0.9357 - val_loss: 0.2809 - val_acc: 0.9142
Epoch 126/300
31/31 [==============================] - 0s 8ms/step - loss: 0.2260 - acc: 0.9102 - val_loss: 0.3476 - val_acc: 0.8643
Epoch 127/300
31/31 [==============================] - 0s 5ms/step - loss: 0.1802 - acc: 0.9370 - val_loss: 0.2508 - val_acc: 0.9470
Epoch 128/300
31/31 [==============================] - 0s 6ms/step - loss: 0.1933 - acc: 0.9290 - val_loss: 0.9290 - val_acc: 0.8518
Epoch 129/300
31/31 [==============================] - 0s 7ms/step - loss: 0.1799 - acc: 0.9478 - val_loss: 0.2480 - val_acc: 0.9423
Epoch 130/300
31/31 [==============================] - 0s 6ms/step - loss: 0.1789 - acc: 0.9404 - val_loss: 0.4339 - val_acc: 0.8783
Epoch 131/300
31/31 [==============================] - 0s 6ms/step - loss: 0.2613 - acc: 0.9022 - val_loss: 0.2864 - val_acc: 0.9017
Epoch 132/300
31/31 [==============================] - 0s 7ms/step - 

31/31 [==============================] - 0s 8ms/step - loss: 0.1720 - acc: 0.9404 - val_loss: 0.2964 - val_acc: 0.9080
Epoch 187/300
31/31 [==============================] - 0s 7ms/step - loss: 0.1508 - acc: 0.9511 - val_loss: 0.8624 - val_acc: 0.8580
Epoch 188/300
31/31 [==============================] - 0s 6ms/step - loss: 0.1618 - acc: 0.9498 - val_loss: 0.2872 - val_acc: 0.9267
Epoch 189/300
31/31 [==============================] - 0s 8ms/step - loss: 0.1366 - acc: 0.9591 - val_loss: 0.3124 - val_acc: 0.9189
Epoch 190/300
31/31 [==============================] - 0s 8ms/step - loss: 0.1159 - acc: 0.9645 - val_loss: 0.2973 - val_acc: 0.9314
Epoch 191/300
31/31 [==============================] - 0s 7ms/step - loss: 0.1837 - acc: 0.9444 - val_loss: 0.3535 - val_acc: 0.9048
Epoch 192/300
31/31 [==============================] - 0s 8ms/step - loss: 0.1287 - acc: 0.9632 - val_loss: 0.7658 - val_acc: 0.8596
Epoch 193/300
31/31 [==============================] - 0s 6ms/step - loss: 0.1824 -

...

31/31 [==============================] - 3s 106ms/step - loss: 0.0390 - acc: 0.9859 - val_loss: 0.1734 - val_acc: 0.9423
Epoch 124/300
31/31 [==============================] - 3s 95ms/step - loss: 0.0390 - acc: 0.9879 - val_loss: 0.1761 - val_acc: 0.9548
Epoch 125/300
31/31 [==============================] - 3s 92ms/step - loss: 0.0809 - acc: 0.9739 - val_loss: 0.1753 - val_acc: 0.9438
Epoch 126/300
31/31 [==============================] - 3s 90ms/step - loss: 0.0420 - acc: 0.9846 - val_loss: 0.1846 - val_acc: 0.9548
Epoch 127/300
31/31 [==============================] - 3s 91ms/step - loss: 0.0481 - acc: 0.9839 - val_loss: 0.1835 - val_acc: 0.9532
Epoch 128/300
31/31 [==============================] - 3s 90ms/step - loss: 0.0337 - acc: 0.9906 - val_loss: 0.1821 - val_acc: 0.9610
Epoch 129/300
31/31 [==============================] - 3s 90ms/step - loss: 0.0361 - acc: 0.9906 - val_loss: 0.2272 - val_acc: 0.9485
Epoch 130/300
31/31 [==============================] - 3s 89ms/step - loss:

Epoch 184/300
31/31 [==============================] - 3s 110ms/step - loss: 0.0093 - acc: 0.9987 - val_loss: 0.1983 - val_acc: 0.9563
Epoch 185/300
31/31 [==============================] - 3s 99ms/step - loss: 0.0170 - acc: 0.9953 - val_loss: 0.2073 - val_acc: 0.9548
Epoch 186/300
31/31 [==============================] - 3s 107ms/step - loss: 0.0167 - acc: 0.9940 - val_loss: 0.2353 - val_acc: 0.9563
Epoch 187/300
31/31 [==============================] - 3s 98ms/step - loss: 0.0158 - acc: 0.9967 - val_loss: 0.2329 - val_acc: 0.9594
Epoch 188/300
31/31 [==============================] - 3s 105ms/step - loss: 0.0151 - acc: 0.9940 - val_loss: 0.2027 - val_acc: 0.9485
Epoch 189/300
31/31 [==============================] - 3s 97ms/step - loss: 0.0096 - acc: 0.9967 - val_loss: 0.2118 - val_acc: 0.9485
Epoch 190/300
31/31 [==============================] - 3s 112ms/step - loss: 0.0157 - acc: 0.9946 - val_loss: 0.2536 - val_acc: 0.9548
Epoch 191/300
31/31 [==============================] - 3s 

Epoch 245/300
31/31 [==============================] - 3s 94ms/step - loss: 0.0090 - acc: 0.9973 - val_loss: 0.2444 - val_acc: 0.9532
Epoch 246/300
31/31 [==============================] - 3s 90ms/step - loss: 0.0167 - acc: 0.9933 - val_loss: 0.2045 - val_acc: 0.9532
Epoch 247/300
31/31 [==============================] - 3s 92ms/step - loss: 0.0164 - acc: 0.9960 - val_loss: 0.2406 - val_acc: 0.9548
Epoch 248/300
31/31 [==============================] - 3s 102ms/step - loss: 0.0150 - acc: 0.9953 - val_loss: 0.2080 - val_acc: 0.9548
Epoch 249/300
31/31 [==============================] - 4s 121ms/step - loss: 0.0186 - acc: 0.9933 - val_loss: 0.2010 - val_acc: 0.9548
Epoch 250/300
31/31 [==============================] - 3s 94ms/step - loss: 0.0133 - acc: 0.9967 - val_loss: 0.2850 - val_acc: 0.9470
Epoch 251/300
31/31 [==============================] - 3s 93ms/step - loss: 0.0118 - acc: 0.9967 - val_loss: 0.2488 - val_acc: 0.9548
Epoch 252/300
31/31 [==============================] - 3s 93

[I 2023-07-09 21:41:58,154] Trial 79 finished with value: 0.9578782916069031 and parameters: {'num_layers': 4, 'dense_neurons': 2057, 'dropout': 3, 'learning_rate': 0.007}. Best is trial 26 with value: 0.9656786322593689.


Trial 0: Accuracy = 0.9656786322593689
Trial 0: Accuracy = 0.9656786322593689
{'num_layers': 4, 'dense_neurons': 1543, 'dropout': 2, 'learning_rate': 0.007}
